# **CAPITULO III: Uso de la directiva for**

La directiva for identifica una construcción iterativa de uso compartido que especifica que las iteraciones del bucle asociado se ejecutarán en paralelo. Las iteraciones del bucle for se distribuyen entre los subprocesos existentes en el equipo que ejecuta la construcción paralela a la que se enlaza. La sintaxis de la construcción for es la siguiente: 

```c++
#pragma omp for [clause[[,] clause] ... ] new-line for-loop
```

La directiva for aplica restricciones a la estructura del bucle for correspondiente. En concreto, el bucle correspondiente for debe tener una forma canónica:

```c++
for (init-expr;var logical-op b;incr-expr)
```

Veamos un ejemplo:
```c++
#pragma omp for 
for (int k = 0; k < N; k++){
    // Code
}
```
Para ver como funciona vamos a imprimir numeros en pantalla con la directiva for y ver como cada iteracion esta asignada a cada subproceso(hilo). 

In [24]:
%%writefile directivaFor.cpp
#include <stdio.h>
#include <iostream>
#include <omp.h>

int main(int argc, char *argv[]){
    int n, threads;
    if (argc > 1) 
    {
        //-- # número de elementos del arreglo
        n = std::stoi(argv[1]);
        //-- # número de hilos de region paralela
        threads = std::stoi(argv[2]);
        //-- region  paralela de threads hilos
        #pragma omp parallel num_threads(threads)
        {
            #pragma omp for
            for (int k = 0; k < n; k++){
                printf("Soy el hilo %d y ejecuto la iteración %d\n", omp_get_thread_num(), k);
            }
        }
        return 0;
    }
    printf("No se encontro Parametro ");
    return 0;
}

Overwriting directivaFor.cpp


Ejecutamos el anterior codigo con **n = 12 y p = 4**

In [25]:
!g++ -fopenmp directivaFor.cpp
!a.exe 12 4

Soy el hilo 0 y ejecuto la iteración 0
Soy el hilo 3 y ejecuto la iteración 9
Soy el hilo 0 y ejecuto la iteración 1
Soy el hilo 2 y ejecuto la iteración 6
Soy el hilo 0 y ejecuto la iteración 2
Soy el hilo 3 y ejecuto la iteración 10
Soy el hilo 2 y ejecuto la iteración 7
Soy el hilo 3 y ejecuto la iteración 11
Soy el hilo 1 y ejecuto la iteración 3
Soy el hilo 1 y ejecuto la iteración 4
Soy el hilo 1 y ejecuto la iteración 5
Soy el hilo 2 y ejecuto la iteración 8


Como vemos cada subproceso ejecuta 3 iteraciones ya que **n = 12**(tamaño del for) y **p = 4** (número de hilos)

veamos que pasa con un **p** que no es múltiplo de **n**, probemos con **n = 15 y p = 4** .


In [26]:
!g++ -fopenmp directivaFor.cpp
!a.exe 15 4

Soy el hilo 1 y ejecuto la iteración 4
Soy el hilo 3 y ejecuto la iteración 12
Soy el hilo 2 y ejecuto la iteración 8
Soy el hilo 3 y ejecuto la iteración 13
Soy el hilo 2 y ejecuto la iteración 9
Soy el hilo 3 y ejecuto la iteración 14
Soy el hilo 2 y ejecuto la iteración 10
Soy el hilo 2 y ejecuto la iteración 11
Soy el hilo 0 y ejecuto la iteración 0
Soy el hilo 0 y ejecuto la iteración 1
Soy el hilo 1 y ejecuto la iteración 5
Soy el hilo 0 y ejecuto la iteración 2
Soy el hilo 1 y ejecuto la iteración 6
Soy el hilo 1 y ejecuto la iteración 7
Soy el hilo 0 y ejecuto la iteración 3


* El hilo p = 0 ejecuta 4 iteraciones
* El hilo p = 1 ejecuta 4 iteraciones
* El hilo p = 2 ejecuta 4 iteraciones
* El hilo p = 3 ejecuta 3 iteraciones

Suma de elementos de un arreglo: para este ejemplo generaremos un arreglo de tamaño n con valores aleatorios entre 1 y 10 inclusive.

In [47]:
%%writefile sumaParalela.cpp
#include <stdio.h>
#include <iostream>
#include <omp.h>
#include <cstdlib>
#include <time.h>

#define max_elementos 100

void VectorAleatorio(int arr[], int tam){
    srand((unsigned) time(NULL));
    for (int k = 0; k < tam; k++){
        arr[k] = 1 + rand() % 10;
    }
}

void MostrarVector(int arr[], int tam){
    printf("[");
    for (int k = 0; k < tam; k++){
        if (k == tam - 1){
            printf("%d", arr[k]);
        }
        else {
            printf("%d ,", arr[k]);
        }
    }
    printf("]\n");
}

int main(int argc, char *argv[]){
    int n, threads;
    if (argc > 1) 
    {
        //-- # número de elementos del arreglo
        n = std::stoi(argv[1]);
        
        //-- # número de hilos de region paralela
        threads = std::stoi(argv[2]);
        
        //-- # inicializamos vector con un nuúmero máximo de elementos y valores aleatorios (0 <= n <= max_elementos)
        int arreglo[max_elementos];
        VectorAleatorio(arreglo, n);
        
        //-- # mostramos el vector aleatorio
        printf("los elementos del vector son: ");
        MostrarVector(arreglo, n);
        
        //-- # suma global
        int suma = 0;
        
        //-- region  paralela de threads hilos
        #pragma omp parallel num_threads(threads)
        {
            #pragma omp for
            for (int k = 0; k < n; k++){
                suma += arreglo[k];
            }
        }
        
        //-- # mostramos la suma de los elementos del arreglo
        printf("la suma es: %d\n", suma);
        return 0;
    }
    printf("No se encontro Parametro\n");
    return 0;
}

Overwriting sumaParalela.cpp


In [48]:
!g++ -fopenmp sumaParalela.cpp
!a.exe 10 2

los elementos del vector son: [2 ,6 ,2 ,4 ,3 ,7 ,8 ,4 ,10 ,1]
la suma es: 47
